In [1]:
import seaborn as sns
import scipy
import skimage
sns.set_style("ticks")
from pathlib import Path

# Get the data 

In [2]:
# in this file is the url to the spreadsheet with the data
# PIGLEG_SPREADSHEET_URL=https://docs.google.com/spreadsheets/d/1G55kXxcJ0...piLaK0snddRU-jzU/edit?usp=sharing
env_paths = list(Path(".").glob("../**/.env.dev"))
print(env_paths)
# find line with PIGLEG_SPREADSHEET_URL and give me the value after the "="
PIGLEG_SPREADSHEET_URL = [line.split("=", 1)[1].strip() for line in open(env_paths[0], "r") if line.startswith("PIGLEG_SPREADSHEET_URL")][0]
PIGLEG_SPREADSHEET_URL = PIGLEG_SPREADSHEET_URL.replace("\"", "")
PIGLEG_SPREADSHEET_URL = PIGLEG_SPREADSHEET_URL.replace("edit?usp=sharing", "export?format=csv")
# PIGLEG_SPREADSHEET_URL

[PosixPath('../.env.dev')]


In [3]:
import pandas as pd
url = PIGLEG_SPREADSHEET_URL
df = pd.read_csv(url)

In [4]:
df.tail()

,filename,report_url,timestamp,email,uploaded_at,finished_at,fps,Needle holder length [m],Scissors length [m],Forceps length [m],...,Scissors stitch 0 length [m],Scissors stitch 0 visibility [s],Scissors stitch 0 velocity,Scissors stitch 0 unit,Scissors stitch 0 visibility [%],Scissors stitch 0 area presence [%],Stitch 1 duration [s],Stitch 1 duration [%],Stitch 2 duration [s],Stitch 2 duration [%]
892,Bad-trim_0B861D6D_C942_4D0E_90C7_5D9779BE091E.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,2024-01-16 12:34:56,Khalatbari.sadaf@gmail.com,2022-08-01 08:47,2024-01-16 11:34,25.0,"2,864749892",0,"0,1863840203",...,0,"0,1333333333",0,m,"0,1813236627",50,NaN,NaN,NaN,NaN
893,Bad-03_11_21_B_Hopfe_Johanna_Einzelknopfnaht.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,2024-01-16 12:40:27,khalatbari.sadaf@gmail.com,2022-07-07 17:33,2024-01-16 11:40,25.0,"5,688298479","0,2870924548","2,515599076",...,"0,03205866509","34,2","0,0007500032432",m,"33,09677419","0,9746588694","156,9333333","50,74369476",NaN,NaN
894,Good-162880einzelknopf.mp4,http://suran.kky.zcu.cz:8000//uploader/web_rep...,2024-01-16 12:43:36,Khalatbari.sadaf@gmail.com,2022-08-01 08:49,2024-01-16 11:43,60.0,"11,83584973","0,7391541411","1,278647685",...,"0,02328418669","40,66666667","0,0005735021351",m,100,0,"35,73333333","6,513549642","80,73333333","14,71624742"
895,Bad-3_11_21_B_Grumt_Alina_Einzelknopfnaht_Schw...,http://suran.kky.zcu.cz:8000//uploader/web_rep...,2024-01-16 12:45:50,khalatbari.sadaf@gmail.com,2022-07-07 17:16,2024-01-16 11:45,25.0,"5,886226351","0,3555818928","0,9973024274",...,"0,1282818914","48,46666667","0,001183155586",m,"40,88863892","0,6877579092",NaN,NaN,NaN,NaN
896,Bad-2021_11_03_A_Noll_Alexander_Einzelknopfnah...,http://suran.kky.zcu.cz:8000//uploader/web_rep...,2024-01-16 12:46:59,khalatbari.sadaf@gmail.com,2022-07-07 17:08,2024-01-16 11:46,25.0,"9,503007304","0,5081350784","1,944407215",...,"0,2747823882","75,6","0,002884251465",m,"86,69724771",0,NaN,NaN,NaN,NaN


In [5]:
# remove all records with filename NA or NaN
dfs = df[df["filename"].notna()]

# take all records with filename containing "Good", "Medium", "Bad", if the filename is the same, take the last one

dfs = dfs[dfs["filename"].str.contains("Good|Medium|Bad")].drop_duplicates(subset=["filename"], keep="last")

In [6]:
# add column "stars_auto" with 0 for "Good", 1 for "Medium", 2 for "Bad"
dfs["stars_auto"] = dfs["filename"].str.contains("Good").astype(int) * 0
dfs["stars_auto"] = dfs["filename"].str.contains("Medium").astype(int) * 1
dfs["stars_auto"] = dfs["filename"].str.contains("Bad").astype(int) * 2
# dfs["filename"].str.contains("Good|Medium|Bad")


In [24]:
len(dfs)

22

In [25]:
dfs.columns

Index(['filename', 'report_url', 'timestamp', 'email', 'uploaded_at',
       'finished_at', 'fps', 'Needle holder length [m]', 'Scissors length [m]',
       'Forceps length [m]',
       ...
       'Scissors stitch 0 visibility [s]', 'Scissors stitch 0 velocity',
       'Scissors stitch 0 unit', 'Scissors stitch 0 visibility [%]',
       'Scissors stitch 0 area presence [%]', 'Stitch 1 duration [s]',
       'Stitch 1 duration [%]', 'Stitch 2 duration [s]',
       'Stitch 2 duration [%]', 'stars_auto'],
      dtype='object', length=444)

### Replace "," with "."

In [26]:
# find the columns where its values contain single "," and digits
dfs.columns[dfs.apply(lambda x: (x.dtype is str) and (x.str.contains(",\d").any()))]
cols = []
for col in dfs.columns:
    try:
        if (dfs[col].str.contains(",\d").any()):
            # convert , to . and change the column type to float
            dfs[col] = dfs[col].str.replace(",", ".").astype(float)
            cols.append(col)
    except:
        pass
# cols

In [27]:
# dfs

In [28]:
# print name of columns containing needle holder
# dfs.columns[dfs.columns.str.contains("Needle holder")]

In [29]:
# dfs[["Needle holder stitch 1 length [m]", "Needle holder length [m]"]]

### If there is just one stitch, put ths information into stitch 1 measurements

In [30]:
# If column containing " stitch 1" in its name has empty values, fill them with  the values from the column with the same name but withuou " stitch 1"
for col in dfs.columns[dfs.columns.str.contains(" stitch 0")]:
    dfs[col] = dfs[col].fillna(dfs[col.replace(" stitch 0", "")])

In [31]:
# dfs[["Needle holder stitch 1 length [m]", "Needle holder length [m]"]]


# Find the correlation

In [15]:
# dfs.corr()

In [16]:
# find corelation between stars_auto and other numerical columns and sort them
dfs.corr()["stars_auto"].sort_values()



annotation_annotation_stars      -0.944911
Stitches parallelism score [%]   -0.664638
Scissors stitch 1 velocity       -0.478465
Scissors velocity                -0.461484
qr_data_size_by_scene            -0.333333
                                    ...   
annotation_annotator                   NaN
frame_width                            NaN
frame_height                           NaN
Stitch 2 duration [s]                  NaN
Stitch 2 duration [%]                  NaN
Name: stars_auto, Length: 206, dtype: float64

In [17]:
# try to find the best columns to predict stars_auto
corr = dfs.corr()["stars_auto"].sort_values(ascending=False).head(10)
corr


stars_auto                               1.000000
Stitch 1 duration [s]                    1.000000
Stitch 1 duration [%]                    1.000000
Scissors bbox stitch 1 length [m]        0.688679
Scissors bbox stitch 0 length [m]        0.669492
Stitches perpendicular score [%]         0.647946
Needle holder stitch 0 length [m]        0.614731
Needle holder stitch 1 length [m]        0.580418
Scissors bbox length [m]                 0.514664
Needle holder stitch 1 visibility [s]    0.506560
Name: stars_auto, dtype: float64

In [35]:
columns = ["filename", "Needle holder stitch 1 length [m]", "Stitches perpendicular score [%]", "Needle holder stitch 1 visibility [s]", "Stitches parallelism score [%]"]

dfs[columns]

,filename,Needle holder stitch 1 length [m],Stitches perpendicular score [%],Needle holder stitch 1 visibility [s],Stitches parallelism score [%]
870,Good-03_11_2021_B_Holzmu_ller_Katharina_Einzel...,NaN,NaN,NaN,NaN
871,Good-2021_11_03_B_Zu_hl_Amelie_Einzelknopfnaht...,NaN,NaN,NaN,NaN
872,Medium-2021_11_03_A_Seifert_Johanna_Einzelknop...,NaN,NaN,NaN,NaN
874,Medium-2021_11_03_A_Tengler_Sophie_Einzelknopf...,NaN,NaN,NaN,NaN
875,Medium-2021_10_12_B_Westermann_Lammers_Johanna...,NaN,NaN,NaN,NaN
876,Good-Einzelknopfnaht_Stoklossa.mp4,NaN,NaN,NaN,NaN
877,Good-2021_11_17_A_Gramlich_Judith_Stresstest_E...,NaN,NaN,NaN,NaN
878,Good-2021_10_27_B_Etzel_Peter_Einzelknopfnaht_...,NaN,NaN,NaN,NaN
879,Bad-2021_11_3_A_Brodersen___nne_Einzelknopfnah...,NaN,NaN,NaN,NaN
880,Bad-2021_11_03_B_V_lz__Ann_Sophie_Einzelknopfn...,NaN,NaN,NaN,NaN


# Train predictor

In [33]:
import sklearn
# train a model to predict stars_auto
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# import SVM
from sklearn.svm import SVR
#train a model to predict stars_auto

# columns = list(corr.index)
columns.append("filename")

dfsx = dfs[columns]
print(len(dfsx))
dfsx = dfsx.dropna()
print(len(dfsx))
X = dfsx[corr.index].drop("stars_auto", axis=1)
y = dfsx["stars_auto"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = SVR()
model.fit(X_train, y_train)

# 
# X

22
10


KeyError: "['stars_auto', 'Stitch 1 duration [s]', 'Stitch 1 duration [%]', 'Scissors bbox stitch 1 length [m]', 'Scissors bbox stitch 0 length [m]', 'Needle holder stitch 0 length [m]', 'Scissors bbox length [m]'] not in index"

In [22]:
dfsx

,stars_auto,Stitch 1 duration [s],Stitch 1 duration [%],Scissors bbox stitch 1 length [m],Scissors bbox stitch 0 length [m],Stitches perpendicular score [%],Needle holder stitch 0 length [m],Needle holder stitch 1 length [m],Scissors bbox length [m],Needle holder stitch 1 visibility [s],filename
893,2,156.933333,50.743695,2.141051,1.166848,100.000000,2.151528,3.420839,3.526894,125.866667,Bad-03_11_21_B_Hopfe_Johanna_Einzelknopfnaht.mp4
894,0,35.733333,6.513550,0.052302,0.157242,73.333333,0.990121,0.098898,0.999684,27.600000,Good-162880einzelknopf.mp4


## Score

In [19]:
model.score(X_test, y_test)

/home/appuser/.local/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


nan

In [20]:
model.score(X_train, y_train)

/home/appuser/.local/lib/python3.8/site-packages/sklearn/metrics/_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


nan

## Check the předictions

In [21]:
y_test

816    2
842    0
840    0
822    2
Name: stars_auto, dtype: int64

In [22]:
y_pred = model.predict(X_test)
y_pred

array([0.12636363, 0.11648296, 0.03380476, 0.10143237])

In [23]:
model.predict(X_train)


array([0.10069563, 0.11084839, 0.09947057, 0.10949899, 0.10615264,
       0.09285266, 0.12814825, 0.11112511, 0.12869557, 0.09870658,
       0.10005963, 0.10915682, 0.10023484, 0.10139677, 0.10023488,
       0.08063209])

In [24]:
y_train

829    0
826    2
833    0
824    2
843    0
841    0
835    2
823    2
830    0
857    2
825    0
834    2
828    0
832    0
839    0
827    0
Name: stars_auto, dtype: int64

In [25]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.5

In [26]:
clf.predict(X_train)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [65]:
y_test

816    2
842    0
840    0
822    2
Name: stars_auto, dtype: int32